In [ ]:
import gurobipy as gp
import math
import pandas as pd

In [ ]:
IP = 0
MIP = 1
LP = 2


# model "Binary Representation of Both Factors" method
def model_both_factors_binary(N, model_type, m):
    # find smallest integer k such that N <= 2^k
    k = 0
    num = 1
    while num < N:
        num *= 2
        k += 1
        
        
    # Add variables
    if model_type == IP:
        X = m.addVars(1, k, vtype=gp.GRB.BINARY, name="X")
        Y = m.addVars(1, k, vtype=gp.GRB.BINARY, name="Y")
        Z = m.addVars(k, k, vtype=gp.GRB.BINARY, name="Z")
    elif model_type == MIP:
        X = m.addVars(1, k, vtype=gp.GRB.BINARY, name="X")
        Y = m.addVars(1, k, vtype=gp.GRB.BINARY, name="Y")
        Z = m.addVars(k, k, vtype=gp.GRB.CONTINUOUS, name="Z")
    elif model_type == LP:
        X = m.addVars(1, k, vtype=gp.GRB.CONTINUOUS, name="X")
        Y = m.addVars(1, k, vtype=gp.GRB.CONTINUOUS, name="Y")
        Z = m.addVars(k, k, vtype=gp.GRB.CONTINUOUS, name="Z")

        
    # Set objective
    m.setObjective(0, gp.GRB.MAXIMIZE)

    
    # Add constraints
    
    # build main constraint
    Z_sum = 0
    for i in range(k):
        for j in range(k):
            Z_sum += Z[i,j] * 2**(i+j)

    m.addConstr(N == Z_sum)
    
    # constraints for each variable Z_{i,j}
    for i in range(k):
        for j in range(k):
            m.addConstr(2*Z[i,j] <= X[0,i] + Y[0,j])
            m.addConstr(X[0,i] + Y[0,j] - 1 <= Z[i,j])
            
    # constraints to prevent trivial factorization
    X_prevent = 0
    Y_prevent = 0
    for i in range(1, k):
        X_prevent += X[0,i]
        Y_prevent += Y[0,i]
    
    m.addConstr(X_prevent >= 1)
    m.addConstr(Y_prevent >= 1)
    
    # extra constraints for LP relaxation
    if model_type == MIP:
        for i in range(k):
            for j in range(k):
                m.addConstr(0 <= Z[i,j])
                m.addConstr(Z[i,j] <= 1)
    elif model_type == LP:
        for i in range(k):
            m.addConstr(0 <= X[0,i])
            m.addConstr(X[0,i] <= 1)
            m.addConstr(0 <= Y[0,i])
            m.addConstr(Y[0,i] <= 1)
            for j in range(k):
                m.addConstr(0 <= Z[i,j])
                m.addConstr(Z[i,j] <= 1)
    
    
    return k
    

In [ ]:
current_type = LP
start = 1000000
end = start + 300
data = []

for N in range(start, end+1):
    m = gp.Model()
    m.Params.LogToConsole = 0
    m.Params.Threads = 2
    
    k = model_both_factors_binary(N, current_type, m)
    
    for i in range(10):
        m.reset(1)
        m.optimize()

        if m.status == gp.GRB.OPTIMAL:
            # get values of X and Y
            power = 1
            factorX = 0
            factorY = 0
            for j in range(k):
                factorX += power * m.getVarByName(f"X[0,{j}]").x
                factorY += power * m.getVarByName(f"Y[0,{j}]").x
                power *= 2
            
            
            correct = 0
            if N == factorX * factorY:
                correct = 1


            data.append([N, factorX, factorY, correct, \
                         m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])
        elif m.status == gp.GRB.INFEASIBLE:
            data.append([N, "INFEASIBLE", "INFEASIBLE", "N/A", \
                         m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])

In [ ]:
df = pd.DataFrame(data, columns=["N", "Factor X", "Factor Y", "Is Correct", \
                                 "Num Vars", "Num Integer Vars", "Num Binary Vars", "Num Constraints", \
                                 "Node Count", "Simplex Iterations Count", "Runtime", "Is MIP"])

In [ ]:
print(df)